# 二维卷积

In [1]:
import torch
import torch.nn as nn

## 二维互相关运算

对于卷积网络（CNN）中的的卷积（convolution）运算，通常表示为互相关（cross-correlation）运算。

**一个二维输入数组和一个二维核数组通过互相关运算输出一个二维数组**。

In [2]:
# no padding and one channel
def corr2d(x, k):
    """
    """
    h, w = k.shape
    output_shape = (x.shape[0] - h + 1, x.shape[1] - w + 1) # no padding
    
    y = torch.zeros(output_shape)
    
    for i in range(y.shape[0]):
        for j in range(y.shape[1]):
            y[i, j] = (x[i:i+h, j:j+w] * k).sum()
    
    return y  


In [3]:
x = torch.arange(16).view(4, 4)
k = torch.ones(3, 3)
y = corr2d(x, k)
print(x)
print(k)
print(y)

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]])
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
tensor([[45., 54.],
        [81., 90.]])


## 二维卷积层

In [4]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super(Conv2D, self).__init__()
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
        
    def forward(self, x):
        
        return corr2d(x, self.weight) + self.bias

In [5]:
x = torch.arange(25).view(5, 5)
kernel_size = (3, 3)

conv2d = Conv2D(kernel_size)
y = conv2d(x)
print(y)

tensor([[ 1.4948,  1.1370,  0.7792],
        [-0.2942, -0.6520, -1.0098],
        [-2.0832, -2.4410, -2.7988]], grad_fn=<AddBackward0>)


## 边缘检测应用

In [6]:
# data
x = torch.ones(10, 10)
# x = torch.ones(6, 8)
# x[:, 2:6] = 0.
x[:, 2:8] = 0.
print(x)

tensor([[1., 1., 0., 0., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 0., 0., 1., 1.]])


In [7]:
edge_kernel = torch.tensor([[-1, 1]])
y = corr2d(x, edge_kernel)
print(y)

tensor([[ 0., -1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.]])


In [8]:
def mse_loss(y_pred, y):
    
    return (y_pred - y.view(y_pred.size())) ** 2 

In [9]:
def sgd(params, lr):
    for param in params:
        param.data -= lr * param.grad

In [10]:
steps = 30
lr = 0.005 # for x.shape = (10, 10)
# lr = 0.01  # for x.shape = (6, 8)
kernel_size = [1, 2]
net = Conv2D(kernel_size)
loss = mse_loss
optimizer = sgd
params = [net.weight, net.bias]

for step in range(steps):
    y_pred = net(x)
    l = loss(y_pred, y).sum()
    
    
    # grad clearing
    for param in params:
        if param.grad is not None:
            # param.grad.fill_(0)
            param.grad.data.zero_()
            

    # computer grad
    l.backward()
    
    # update grad
    optimizer(params, lr)
    
    if (step + 1) % 5 == 0:
        print('step {}, loss {:.3f}'.format(step + 1, l.item()))


step 5, loss 3.980
step 10, loss 1.215
step 15, loss 0.411
step 20, loss 0.142
step 25, loss 0.050
step 30, loss 0.017


**当损失函数值没有降低， 即模型训练不能收敛时，主要原因是因为学习率过大。因此可以通过减小学习率的大小，找到合适的学习率，使得模型收敛**

In [11]:
print(net.weight.data)

tensor([[-0.9734,  0.9736]])
